In [1]:
import cv2
import datetime
import time
from pathlib import Path

In [2]:
#ディレクトリの変更
import os
os.chdir(r"C:\Users\ara-d\pokemon_analisis")

In [3]:
from pywinauto.application import Application

In [4]:
#参考：「[Python]マルチモニター環境でのウィンドウキャプチャ」
#https://qiita.com/danupo/items/e196e0e07e704796cd42
import win32gui
import win32ui
import win32con
import numpy as np
import cv2

##アクティブなウィンドウを探して画像配列（numpy array)を返す様な関数を作る
def WindowCapture(window_name: str, bgr2rgb: bool = False):
    # 現在アクティブなウィンドウ名を探す
    process_list = []

    def callback(handle, _):
        process_list.append(win32gui.GetWindowText(handle))
    win32gui.EnumWindows(callback, None)

    #print(process_list)

    # ターゲットウィンドウ名を探す
    for process_name in process_list:
        if window_name in process_name:
            hnd = win32gui.FindWindow(None, process_name)
            #print("found") #見つかったら出力
            break
    else:
        # 見つからなかったら画面全体を取得
        #print("not found") #見つからなかったら出力
        hnd = win32gui.GetDesktopWindow()

    # ウィンドウサイズ取得
    x0, y0, x1, y1 = win32gui.GetWindowRect(hnd)
    width = x1 - x0
    height = y1 - y0
    # ウィンドウのデバイスコンテキスト取得
    windc = win32gui.GetWindowDC(hnd)
    srcdc = win32ui.CreateDCFromHandle(windc)
    memdc = srcdc.CreateCompatibleDC()
    # デバイスコンテキストからピクセル情報コピー, bmp化
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)
    memdc.SelectObject(bmp)
    memdc.BitBlt((0, 0), (width, height), srcdc, (0, 0), win32con.SRCCOPY)

    # bmpの書き出し
    if bgr2rgb is True:
        img = np.frombuffer(bmp.GetBitmapBits(True), np.uint8).reshape(height, width, 4)
        img = cv2.cvtColor(img, cv2.COLOR_bgr2rgb)
    else:
        img = np.fromstring(bmp.GetBitmapBits(True), np.uint8).reshape(height, width, 4)

    # 後片付け
    # srcdc.DeleteDC()
    memdc.DeleteDC()
    # win32gui.ReleaseDC(hnd, windc)
    win32gui.DeleteObject(bmp.GetHandle())

    return img

In [53]:
#画像をとりあえずキャプチャする（全体）
img = WindowCapture("PotPlayer")

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_7728/2713382240.py:50: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(bmp.GetBitmapBits(True), np.uint8).reshape(height, width, 4)


In [54]:
##############ここは一回だけでいい###################
'''
#"Select Time"だけ切り取る
##やってみる
Select = img[0:80,0:400]
cv2.imshow("",Select)
cv2.waitKey(0)
cv2.destroyAllWindows()

##画像データを保存
np.save(
    "imgdata" + "_" + "SelectTime",  # ファイル名
    Select # 保存したいオブジェクト
)
'''

In [56]:
#保存していた"SelectTime"データを解凍する。
SelectTime = np.load(file="imgdata_SelectTime.npy")
cv2.imshow("",SelectTime)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [57]:
'''
cv2.imshow("", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

'\ncv2.imshow("", img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [58]:
#相手のポケモンの画像を抽出する関数を作る。
def capture_opponent(img):
    a,b,c,d = 255, 961, 175, 225
    #相手ポケモンの画像だけ抽出
    part = img[a:b,c:d]
    
    #一体ずつポケモンの画像を抽出
    poke = []
    for i in range(6):
        new_poke = part[int((b - a)/6) * i + 20:int((b - a)/6) * (i + 1) - 20]
        poke.append(new_poke)
    return poke

In [59]:
'''
#各ポケモン画像を確認する。
poke = capture_opponent(img)
for i in poke:
    cv2.imshow("", i)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

'''

'\n#各ポケモン画像を確認する。\npoke = capture_opponent(img)\nfor i in poke:\n    cv2.imshow("", i)\n    cv2.waitKey(0)\n    cv2.destroyAllWindows()\n\n'

In [60]:
#個別ポケモン画像データ(numpy配列)を保存する関数を作る
##入力は画像データが入っている（６体分）リスト
def save_poke(Img_array):    
    ##今日の日付
    time_ = str(datetime.datetime.now())
    time_ = time_[:19].replace(":", "-").replace(" ", "-") #ファイル名が保存出来るように調整する。

    #ディレクトリを直下の"poke_images"に移動する。
    os.chdir(r"C:\Users\ara-d\pokemon_analisis\poke_images")
    #画像を保存する。
    for i in range(len(Img_array)):     
        np.save(
            "pokedata" + "_" + time_ + "_" + str(i),  # ファイル名
            Img_array[i] # 保存したいオブジェクト
        )

In [61]:
#保存していた名前データを解凍する。
##ディレクトリを指定し、ファイル名を取得する。
path = r"C:\Users\ara-d\pokemon_analisis\poke_names"
dir_ = Path(path)

poke_names_FileName = sorted(dir_.glob("*.npy"))
##ファイル名を全て表示
poke_names_FileName

[WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_names/poke_names_2021-04-19-18-59-27.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_names/poke_names_2021-04-19-20-08-20.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_names/poke_names_2021-04-19-21-54-09.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_names/poke_names_2021-04-19-22-23-08.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_names/poke_names_2021-04-22-00-33-53.npy')]

In [62]:
##名前データを呼び起こす
Names = []
for i in range(len(poke_names_FileName)):
    new = list(np.load(file=str(poke_names_FileName[i])))
    ##データを解凍する。
    ##一致率を計算する。
    Names.extend(new)
    
#名前データを確認する
Names = np.array(Names)
Names

array(['ファイヤー', 'サンダー', 'フリーザー', 'ガラルヒヒダルマ', 'カミツルギ', 'パッチラゴン', 'カイリュー',
       'カプレヒレ', '霊獣ボルトロス', 'ポリゴン２', 'ミミッキュ', 'ナットレイ', 'ユクシー', 'バクガメス',
       'エルレイド', 'ミュウツー', 'ウオノラゴン', 'マホミル', 'ユキハミ', 'メタモン', 'ツチニン',
       'ヨクバリス', 'オトスパス', 'ナマコブシ', 'マーシャドー', '黒バドレックス', 'ザシアン', 'カイオーガ',
       'イベルタル', 'ギャラドス', 'ブリザポス', 'レジドラゴ', 'カメックス', 'ムゲンダイナ', 'ザマゼンタ',
       'キングドラ', 'ニョロトノ', 'ボルトロス', '霊獣ランドロス', 'ザシアン', 'ドラパルト', 'ホウオウ',
       'ガオガエン', 'ウオノラゴン', 'ライチュウ', 'ゴリランダー', 'エルフーン', 'エルフーン', 'ザシアン',
       'ガオガエン', 'レジエレキ', 'ウーラオス', '霊獣ランドロス', 'ミミッキュ', 'ウーラオス', 'ザシアン',
       'ミカルゲ', 'ドラパルト', 'レジギガス', 'ウーラオス', 'カプコケコ', 'ガラルヤドラン', 'サーナイト',
       'フシギバナ', 'ドラパルト', 'ブリムオン', 'リザードン', 'ゾロアーク', 'カメックス', 'フシギバナ',
       'ホウオウ', 'ディアルガ', 'フシギバナ', 'レジエレキ', 'エルフーン', 'コータス', 'ガオガエン',
       'ゼルネアス', 'ルナアーラ', 'ミュウツー', 'ソルガレオ', 'ホウオウ', 'ガラルマッスグマ', 'ギガイアス',
       'ミュウツー', 'エルフーン', 'ポットデス', 'カプテテフ', 'メタモン', 'カプコケコ', 'インテレオン',
       'ガラルサンダー', 'エースバーン', 'ドラパルト', 'ジガルデ', 'ウーラオス', 'ザシアン', 'サンダー',
       'ミミッキュ

In [63]:
#二つの画像データの一致率を計算する関数を作る。
def calc_accuracy(image1, image2): 
    A = image1 == image2
    A = np.reshape(A, (1,-1))
    accuracy = sum(A[0])/len(A[0])
    
    return accuracy

In [64]:
#ポケモン画像を判別し、名前を出力する関数を作る。
def judge_poke(input_image, name_data, train_img_path): #入力：（入力画像データ（numpy array)、名前データ（numpy array、一次元)、train dataがある場所）
    #ディレクトリを指定
    dir_ = Path(train_img_path)
    #".npy"となっているファイルを列挙する。
    poke_images_FileName = sorted(dir_.glob("*.npy"))
    
    Accuracies = []

    #各train dataをチェックし、一致率を計算する。
    for i in range(len(poke_images_FileName)):
        ##データを解凍する。
        poke_image_train = np.load(file=str(poke_images_FileName[i]))
        ##一致率を計算する。
        accuracy = calc_accuracy(input_image, poke_image_train) 
        ##一致率を保存する
        Accuracies.append(accuracy)
        
        #もし一致率が100％であればbreak
        if accuracy == 1:
            break

    #print(Accuracies)
    #Accuracies最大値のindexを計算
    max_index = Accuracies.index(max(Accuracies))

    #ポケモンの名前を引き出す
    return name_data[max_index], max(Accuracies)

In [65]:
battle_limit = 120
waiting_limit = 900

In [66]:
cv2.imshow("",SelectTime)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [67]:
cv2.imshow("",img[0:80,0:400])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
SelectTime.shape

(80, 400, 4)

In [42]:
sum(sum(sum(img[0:80,0:400] == SelectTime))) / (80 * 400 * 4)

0.34271875

In [31]:
img[0:80,0:400].shape

(80, 400, 4)

In [68]:
####################メイン部分#####################
####画像を自動でキャプチャ、読み込み画像判別&学習用に保存

#trainデータ（画像情報が入ったnumpy array）がある場所を指定する。
train_img_path = r"C:\Users\ara-d\pokemon_analisis\poke_images_train"

count = 0
while True:
    #画像をキャプチャする。
    img = WindowCapture("PotPlayer") # 部分一致
    if sum(sum(sum(img[0:80,0:400] == SelectTime))) / (80 * 400 * 4) >= 0.8:
        print("Battle Start")
        #画像を出力
        #cv2.imshow("", img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        
        #個別のポケモンの画像を抽出する。
        poke = capture_opponent(img)
        
        #ここの部分は判別用
        for individual_image in poke:
            print(judge_poke(individual_image, Names, train_img_path))
        
        ##個別ポケモン画像データを保存する。
        save_poke(poke)
        
        count = 0 #カウントをリフレッシュする。
        
        #対戦時間中(battle_limit中)は休憩
        for i in range(battle_limit // 30):
            print("Battle Time Remain:" + str(battle_limit - 30 * i)) #30秒毎に警告
            time.sleep(30) 
        else:
            print("Rest was finished. Count starts again.")

    if count % 30 == 0:
        print("Waiting Time Remain:" + str(waiting_limit - count)) #30秒毎に警告
    if count >= waiting_limit: #一定時間（待機時間がwaiting_limitを超えたらbreak、放置対策）
        break
        
    #１秒経過させる。
    time.sleep(1)        
    count += 1

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_7728/2713382240.py:50: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(bmp.GetBitmapBits(True), np.uint8).reshape(height, width, 4)


Battle Start
('ギガイアス', 0.3888961038961039)
('ポリゴン２', 0.3717532467532468)
('リオル', 0.44422077922077924)
('リオル', 0.4666883116883117)
('レジエレキ', 0.41805194805194806)
('ザシアン', 0.6932467532467532)
Battle Time Remain:120
Battle Time Remain:90
Battle Time Remain:60
Battle Time Remain:30
Rest was finished. Count starts again.
Waiting Time Remain:900
Waiting Time Remain:870
Waiting Time Remain:840
Waiting Time Remain:810
Waiting Time Remain:780
Waiting Time Remain:750
Waiting Time Remain:720
Waiting Time Remain:690
Waiting Time Remain:660
Waiting Time Remain:630
Waiting Time Remain:600
Waiting Time Remain:570
Waiting Time Remain:540
Waiting Time Remain:510
Waiting Time Remain:480
Waiting Time Remain:450
Waiting Time Remain:420
Waiting Time Remain:390
Battle Start
('ガラルサニーゴ', 0.5331818181818182)


KeyboardInterrupt: 

In [39]:
tmp = img[0:80,0:400]

cv2.imshow("", tmp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
SelectTime
cv2.imshow("", SelectTime)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
tmp == SelectTime

array([[[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        ...,
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]],

       [[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        ...,
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]],

       [[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        ...,
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]],

       ...,

       [[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        ...,
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]],

       [[ True,

In [19]:
#個別ポケモン画像の確認
for i in poke:
    cv2.imshow("",i)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    #print(judge_poke(i, Names, train_img_path))

NameError: name 'poke' is not defined

In [42]:
###################ポケモンの名前を入力する（学習パ―ト）#############

In [20]:
#ここに絶対パスを入力する。(ポケモン画像ファイルの場所)
path = r"C:\Users\ara-d\pokemon_analisis\poke_images"
dir_ = Path(path)
poke_images_FileName = sorted(dir_.glob("*.npy"))

poke_images_FileName

[WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-16-32_0.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-16-32_1.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-16-32_2.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-16-32_3.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-16-32_4.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-16-32_5.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-31-11_0.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-31-11_1.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-31-11_2.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images/pokedata_2021-04-21-15-31-11_3.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_ana

In [21]:
Poke_names = []

#保存していた個別ポケモン画像データを解凍し、名前を手動で入力していく。入力し終わったら"処理済み"ファイルに移動させておく。
for i in range(len(poke_images_FileName)):
    poke_image = np.load(file=str(poke_images_FileName[i]))
    cv2.imshow("", poke_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    poke_name = input()
    Poke_names.append(poke_name)
Poke_names

ガブリアス
テッカグヤ
スイクン
バシャーモ
ボーマンダ
ザシアン
ポットデス
エースバーン
レジエレキ
アーマーガア
水ロトム
ポリゴン２
カメックス
イベルタル
ゲンガー
ボスゴドラ
ガラルサニーゴ
マッシブーン
エースバーン
オーロンゲ
ザシアン
ノコッチ
ウーラオス
カミツルギ
ムーランド
シロデスナ
ヌメルゴン
エースバーン
クレベース
イベルタル
ザシアン
サンダー
ラプラス
ウーラオス
エースバーン
メタモン
パルシェン
リザードン
キリキザン
オノノクス
ストリンダ―
アマルルガ
カイリュー
ジバコイル
ゴチルゼル
ザシアン
ラプラス
ウーラオス


['ガブリアス',
 'テッカグヤ',
 'スイクン',
 'バシャーモ',
 'ボーマンダ',
 'ザシアン',
 'ポットデス',
 'エースバーン',
 'レジエレキ',
 'アーマーガア',
 '水ロトム',
 'ポリゴン２',
 'カメックス',
 'イベルタル',
 'ゲンガー',
 'ボスゴドラ',
 'ガラルサニーゴ',
 'マッシブーン',
 'エースバーン',
 'オーロンゲ',
 'ザシアン',
 'ノコッチ',
 'ウーラオス',
 'カミツルギ',
 'ムーランド',
 'シロデスナ',
 'ヌメルゴン',
 'エースバーン',
 'クレベース',
 'イベルタル',
 'ザシアン',
 'サンダー',
 'ラプラス',
 'ウーラオス',
 'エースバーン',
 'メタモン',
 'パルシェン',
 'リザードン',
 'キリキザン',
 'オノノクス',
 'ストリンダ―',
 'アマルルガ',
 'カイリュー',
 'ジバコイル',
 'ゴチルゼル',
 'ザシアン',
 'ラプラス',
 'ウーラオス']

In [22]:
#名前の配列も保存しておく
##保存先ディレクトリを変更
os.chdir(r"C:\Users\ara-d\pokemon_analisis\poke_names")

Poke_names = np.array(Poke_names)

time_ = str(datetime.datetime.now())
time_ = time_[:19].replace(":", "-").replace(" ", "-") #ファイル名が保存出来るように調整する。
np.save(
    "poke_names" + "_" + time_,  # ファイル名
    Poke_names # 保存したいオブジェクト
)

In [ ]:
#以下番外

In [37]:
#動画を読み込む
import cv2

#captureの準備
camera = cv2.VideoCapture(r"C:\Users\ara-d\Videos\Captures\Zoom ミーティング 2020-12-15 13-12-01.mp4")

In [129]:
import time
for i in range(0, 100, 1):
    time.sleep(1)
    print(i, "秒経過")

0 秒経過
1 秒経過
2 秒経過
3 秒経過
4 秒経過
5 秒経過
6 秒経過
7 秒経過
8 秒経過
9 秒経過
10 秒経過
11 秒経過
12 秒経過
13 秒経過
14 秒経過
15 秒経過
16 秒経過
17 秒経過
18 秒経過
19 秒経過
20 秒経過
21 秒経過
22 秒経過


KeyboardInterrupt: 

In [135]:
###train dataのサイズ変更を行う。(一度だけ)
'''
train_img_path = r"C:\Users\ara-d\pokemon_analisis\poke_images_train"
dir_ = Path(train_img_path)

#".npy"となっているファイルを列挙する。
poke_images_FileName = sorted(dir_.glob("*.npy"))

for i in poke_images_FileName:
    input_image = np.load(file = str(i))
    a = input_image[20: 97,35:85,:]
    #cv2.imshow("", a)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    np.save(
        str(i),  # ファイル名
        a # 保存したいオブジェクト
    )
    
'''

In [16]:
dir_ = Path(r"C:\Users\ara-d\pokemon_analisis\poke_images_train")
#".npy"となっているファイルを列挙する。
poke_images_FileName = sorted(dir_.glob("*.npy"))

poke_images_FileName

[WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-42-12_0.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-42-12_1.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-42-12_2.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-42-12_3.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-42-12_4.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-42-12_5.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-56-23_0.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-56-23_1.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-18-23-56-23_2.npy'),
 WindowsPath('C:/Users/ara-d/pokemon_analisis/poke_images_train/pokedata_2021-04-1

In [50]:
for i in poke_images_FileName:
    input_image = np.load(file = str(i))
    cv2.imshow("", input_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()